In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../data/" directory

import os
import os.path as osp

date = '20230223'
ds_name = 'resp+TB'
#resp_folder = f'/mnt/data2/chaoyue/data/ddf_resp_20221130/split_rs123' # IN paper the ARI+TB was constructed by resp_20221130
resp_folder = f'/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123'
for dirname, _, filenames in os.walk(resp_folder):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/valid_edge.csv
/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/test.csv
/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/test_edge.csv
/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/train.csv
/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/train_edge.csv
/mnt/data2/chaoyue/data/ddf_resp_20230131/split_rs123/valid.csv


In [2]:
tb_folder = f'/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123'
for dirname, _, filenames in os.walk(tb_folder):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/valid_edge.csv
/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/test.csv
/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/test_edge.csv
/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/train.csv
/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/train_edge.csv
/mnt/data2/chaoyue/data/ddf_TB_20230223/split_rs123/valid.csv


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import norm
#import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import skew, norm
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
%matplotlib inline

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)

In [4]:
random_seed = 123
date='20230222'
np.random.seed(random_seed)
split_folder = f'/mnt/data2/chaoyue/data/ddf_resp+TB_{date}/split_rs{random_seed}'
os.makedirs(split_folder, exist_ok=True)

In [5]:
for dirname, _, filenames in os.walk(resp_folder):
    for filename in filenames:
        TB_df = pd.read_csv(f'{tb_folder}/{filename}')
        resp_df = pd.read_csv(f'{resp_folder}/{filename}')
        TB_df['sim'] = TB_df['sim']+10000
        full_df = pd.concat([resp_df,TB_df],ignore_index=True)
        print(filename)
        print(len(full_df))
        print(len(full_df['sim'].unique()))
        full_df.to_csv(f'{split_folder}/{filename}', index=False)

valid_edge.csv
10257707
3152
test.csv
10265265
3152
test_edge.csv
10262113
3152
train.csv
30734664
9452
train_edge.csv
30725212
9452
valid.csv
10260859
3152


In [9]:
10257707-10260859

-3152

In [10]:
10262113-10265265

-3152

In [11]:
30725212-30734664

-9452

In [12]:
filename = 'train.csv'
TB_df = pd.read_csv(f'{tb_folder}/{filename}')
resp_df = pd.read_csv(f'{resp_folder}/{filename}')
TB_df['sim'] = TB_df['sim']+10000
full_node_df = pd.concat([resp_df,TB_df],ignore_index=True)


In [13]:
filename = 'train_edge.csv'
TB_df = pd.read_csv(f'{tb_folder}/{filename}')
resp_df = pd.read_csv(f'{resp_folder}/{filename}')
TB_df['sim'] = TB_df['sim']+10000
full_df = pd.concat([resp_df,TB_df],ignore_index=True)

In [14]:
train_ids = full_df['sim'].unique()

In [15]:
c1 = 0
c2 = 0
c3 = 0
for onetree_id in train_ids:
    onetree_node_df = full_node_df[full_node_df['sim'] == onetree_id]
    onetree_edge_df = full_df[full_df['sim'] == onetree_id]
    leaves = list(set(onetree_edge_df['new_to'].values)-set(onetree_edge_df['new_from'].values))
    leaf_df = onetree_node_df[onetree_node_df['node'].isin(leaves)]
    c1 += len(leaf_df[leaf_df['dynamic'] == 'static'])
    c2 += len(leaf_df[leaf_df['dynamic'] == 'decay'])
    c3 += len(leaf_df[leaf_df['dynamic'] == 'growth'])

In [16]:
[(c1+c2+c3)/(i*3) for i in [c1,c2,c3]]

[0.3406808892621469, 51.519496230514754, 22.079112613356273]